In [1]:
# You may need to use 'pip3' instead of 'pip' depending on your system setup
! pip install pandas sqlalchemy

'DOSKEY' is not recognized as an internal or external command,
operable program or batch file.


In [1]:
import pandas as pd
import sqlite3
import os

# Configuration
DB_NAME = 'sales_analysis.db'
TABLE_NAME = 'sales_data'
CSV_FILE = r'C:\Users\ABHILASH\Desktop\hcl\FINAL_DATASET.csv' 

# --- 1. Load Data ---
if not os.path.exists(CSV_FILE):
    print(f"Error: CSV file '{CSV_FILE}' not found. Please add your data file.")
    exit()

df = pd.read_csv(CSV_FILE)
print(f"Loaded {len(df)} rows from CSV. Starting insertion...")

# --- 2. Connect and Insert ---
try:
    # Connects to the database file (creates it if it doesn't exist)
    conn = sqlite3.connect(DB_NAME)
    
    # Define SQL data types explicitly for precision (optional but recommended)
    dtype_map = {
        'date': 'TEXT',
        'order_id': 'INTEGER',
        'price': 'REAL',
        'promotion_flag': 'INTEGER',
        'total_sales': 'REAL',
        'branch_id': 'TEXT',
        # ... map other columns similarly using TEXT, INTEGER, REAL
    }
    
    # Write DataFrame to SQL table
    df.to_sql(
        TABLE_NAME, 
        conn, 
        if_exists='replace', # Replace table if it already exists
        index=False,         # Don't save the pandas index as a column
        dtype=dtype_map      # Use the explicit type mapping
    )
    
    print(f"✅ Data successfully inserted into table '{TABLE_NAME}' in '{DB_NAME}'.")

    # --- 3. Run a verification query ---
    verification_df = pd.read_sql(f"SELECT COUNT(*) AS RowCount, SUM(total_sales) AS TotalRevenue FROM {TABLE_NAME};", conn)
    print("\nVerification Check:")
    print(verification_df)

except Exception as e:
    print(f"An error occurred: {e}")
finally:
    if 'conn' in locals() and conn:
        conn.close()

Loaded 3520 rows from CSV. Starting insertion...
✅ Data successfully inserted into table 'sales_data' in 'sales_analysis.db'.

Verification Check:
   RowCount  TotalRevenue
0      3520     213568.86


In [2]:
import pandas as pd

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3520 entries, 0 to 3519
Data columns (total 24 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   date             3520 non-null   object 
 1   product_id       3520 non-null   object 
 2   product_name     3520 non-null   object 
 3   category         3520 non-null   object 
 4   branch_id        3520 non-null   object 
 5   branch_name      3520 non-null   object 
 6   order_id         3520 non-null   int64  
 7   price            3520 non-null   float64
 8   promotion_flag   3520 non-null   int64  
 9   festival_flag    3520 non-null   int64  
 10  holiday_flag     3520 non-null   int64  
 11  seasonal_index   3520 non-null   float64
 12  total_sales      3520 non-null   float64
 13  online_sales     3520 non-null   float64
 14  offline_sales    3520 non-null   float64
 15  returns_count    3520 non-null   float64
 16  cpi              3520 non-null   float64
 17  day_of_week   

In [3]:
eda=df.copy()

In [4]:
eda.head()

,date,product_id,product_name,category,branch_id,branch_name,order_id,price,promotion_flag,festival_flag,...,offline_sales,returns_count,cpi,day_of_week,month,lag_1,lag_7,lag_30,rolling_mean_7,rolling_mean_30
0,2021-01-31,P001,Men_Tshirt,Male,B001,Bangalore,31,58.70,0,1,...,31.97,4.35,150.00,6,1,115.12,130.14,121.02,106.288571,105.737667
1,2021-02-01,P001,Men_Tshirt,Male,B001,Bangalore,32,58.01,0,0,...,20.86,2.64,150.64,0,2,131.32,125.05,112.25,100.662857,104.851667
2,2021-02-02,P001,Men_Tshirt,Male,B001,Bangalore,33,55.17,0,0,...,22.61,3.39,150.64,1,2,85.67,93.74,155.90,100.540000,102.751000
3,2021-02-03,P001,Men_Tshirt,Male,B001,Bangalore,34,58.21,0,0,...,20.74,4.20,150.64,2,2,92.88,93.47,75.15,99.354286,103.085000
4,2021-02-04,P001,Men_Tshirt,Male,B001,Bangalore,35,57.15,0,0,...,24.67,3.43,150.64,3,2,85.17,89.29,91.68,101.071429,103.406000


In [5]:
cat_col = [col for col in eda.columns if eda[col].dtype == 'object']
num_col = [col for col in eda.columns if eda[col].dtype != 'object']

print('Categorical columns:', cat_col)
print('Numerical columns:', num_col)

Categorical columns: ['date', 'product_id', 'product_name', 'category', 'branch_id', 'branch_name']
Numerical columns: ['order_id', 'price', 'promotion_flag', 'festival_flag', 'holiday_flag', 'seasonal_index', 'total_sales', 'online_sales', 'offline_sales', 'returns_count', 'cpi', 'day_of_week', 'month', 'lag_1', 'lag_7', 'lag_30', 'rolling_mean_7', 'rolling_mean_30']


In [6]:
df[cat_col].nunique()

date            220
product_id        4
product_name      4
category          2
branch_id         4
branch_name       4
dtype: int64

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Copying the original dataset
df_encoded = df.copy()

# Label Encoding for Promotion (mostly for target variable)
label_encoder = LabelEncoder()
df_encoded[""] = label_encoder.fit_transform(df_encoded["Promotion"])

df_encoded

In [7]:
CSV_FILE = r'C:\Users\ABHILASH\Desktop\hcl\FINAL_DATASET.csv' 